In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

# Test the Equivalence of Regular Expressions

In this notebook, we test the equivalence algorithm implemented in `09-Equivalence.ipynb`.

While a simple boolean `true`/`false` result is mathematically sufficient, it is often unsatisfying for educational purposes. Therefore, we implement an advanced **Visual Testing Framework** that:

1.  **Minimizes** the resulting automata.
2.  **Visualizes** them side-by-side.
3.  **Generates Counter-Examples** (witnesses) if the expressions are not equivalent.

We leverage the `minimize` function from `07-Minimize` and the `ProductDFA` logic from `09-Equivalence`.

In [ ]:
import { instance } from "@viz-js/viz";
import { RecursiveSet } from "recursive-set";
import { display } from "tslab";

import { Char } from "./01-NFA-2-DFA";
import { RegExp } from "./03-RegExp-2-NFA";
import { parse } from "./RegExp-Parser";

import { 
    regExpEquiv, 
    regexp2DFA, 
    fsm_complement, 
    ProductDFA 
} from "./09-Equivalence";

import { minimize } from "./07-Minimize";
import { dfa2dot } from "./FSM-2-Dot";

const viz = await instance();

## 1. Finding Counter-Examples (Witnesses)

If two regular expressions $r_1$ and $r_2$ are **not** equivalent, there must exist at least one word $w$ such that:
$$w \in L(r_1) \setminus L(r_2) \quad \text{or} \quad w \in L(r_2) \setminus L(r_1)$$

We find the **shortest** such word by performing a **Breadth-First Search (BFS)** on the Difference Automaton (Product Automaton) constructed by `fsm_complement`.
The first accepting state we hit corresponds to the shortest witness $w$.

In [ ]:
function findWitness(F: ProductDFA): string | null {
    const queue: { state: any, word: string }[] = [{ state: F.q0, word: "" }];
    const visited = new RecursiveSet<any>(F.q0);
    
    const getKey = (s: any, c: string) => `${s.toString()},${c}`;

    let head = 0;
    while(head < queue.length) {
        const { state, word } = queue[head++];

        if (F.A.has(state)) {
            return word === "" ? "ε" : word;
        }

        for (const c of F.Sigma) {
            const nextState = F.delta.get(getKey(state, c));
            if (nextState && !visited.has(nextState)) {
                visited.add(nextState);
                queue.push({ state: nextState, word: word + c });
            }
        }
    }
    return null;
}

## 2. Visual Comparison Strategy

To visually verify equivalence, we cannot simply plot the raw DFAs generated from the regular expressions, as they might have different structures despite accepting the same language.

**The Solution: Minimization**
Recall the Myhill-Nerode theorem: The minimal DFA for a regular language is **unique** (up to the renaming of states).

Therefore, our testing strategy is:
1.  Convert $r_1, r_2 \to \text{DFA}_1, \text{DFA}_2$.
2.  Compute $\text{MinDFA}_1 = \text{minimize}(\text{DFA}_1)$ and $\text{MinDFA}_2 = \text{minimize}(\text{DFA}_2)$.
3.  If $r_1 \equiv r_2$, then $\text{MinDFA}_1$ and $\text{MinDFA}_2$ must be **isomorphic** (look identical).

The function `testVisual` implements this pipeline and renders the graphs side-by-side.

In [ ]:
async function testVisual(
  Sigma: RecursiveSet<Char>,
  s1: string,
  s2: string
): Promise<void> {
  console.log(`Checking: "${s1}" vs "${s2}"`);
  
  try {
    const r1 = parse(s1);
    const r2 = parse(s2);

    const dfa1 = regexp2DFA(r1, Sigma);
    const dfa2 = regexp2DFA(r2, Sigma);

    const min1 = minimize(dfa1);
    const min2 = minimize(dfa2);

    const diff1 = fsm_complement(min1, min2); // L(r1) \ L(r2)
    const diff2 = fsm_complement(min2, min1); // L(r2) \ L(r1)
    
    const witness1 = findWitness(diff1);
    const witness2 = findWitness(diff2);
    const equivalent = (witness1 === null && witness2 === null);

    const dot1 = dfa2dot(min1).dot;
    const dot2 = dfa2dot(min2).dot;
    
    const svg1 = viz.renderString(dot1, { format: "svg" });
    const svg2 = viz.renderString(dot2, { format: "svg" });

    if (equivalent) {
        console.log("✅ RESULT: Equivalent!");
        console.log("Both expressions produce isomorphic minimal DFAs.");
    } else {
        console.log("❌ RESULT: NOT Equivalent.");
        if (witness1) console.log(`Counter-example: "${witness1}" is in L(r1) but NOT in L(r2).`);
        if (witness2) console.log(`Counter-example: "${witness2}" is in L(r2) but NOT in L(r1).`);
    }

    display.html(`
    <div style="display: flex; gap: 20px; border: 1px solid #ccc; padding: 10px; align-items: flex-start;">
        <div style="flex: 1; text-align: center;">
            <h3>RegExp 1</h3>
            <code>${s1}</code>
            <div style="margin-top: 10px;">${svg1}</div>
        </div>
        <div style="border-left: 1px solid #ccc;"></div>
        <div style="flex: 1; text-align: center;">
            <h3>RegExp 2</h3>
            <code>${s2}</code>
            <div style="margin-top: 10px;">${svg2}</div>
        </div>
    </div>
    `);

  } catch (e) {
    console.error(`Error:`, e);
  }
}

## 3. Standard Equivalence Testing

First, we define a simple wrapper function `test`. This function is useful for quick verification without graphical overhead. It:
1.  Parses two regular expression strings.
2.  Checks their equivalence using the `regExpEquiv` algorithm.
3.  Logs a text message indicating the result.

In [ ]:
function test(
  Sigma: RecursiveSet<Char>,
  s1: string,
  s2: string
): void {
  try {
    const r1: RegExp = parse(s1);
    const r2: RegExp = parse(s2);

    if (regExpEquiv(r1, r2, Sigma)) {
      console.log(
        `The regular expressions ${s1} and ${s2} are equivalent.`
      );
    } else {
      console.log(
        `The regular expressions ${s1} and ${s2} are not equivalent.`
      );
    }
  } catch (e) {
    console.error(`Error testing equivalence for ${s1} and ${s2}:`, e);
  }
}

We define the alphabet $\Sigma = \{a, b, c\}$ and run several test cases. These examples range from simple identities (like commutativity checks) to complex expressions derived from algorithmic transformations.

In [ ]:
const Sigma = new RecursiveSet<Char>("a", "b", "c");

In [ ]:
test(Sigma, '(ε+a)(a+ε)*(a+ε)', 'a*');

In [ ]:
test(Sigma, "(ba)(ba)*", "b(ab)*a");

In [ ]:
test(Sigma, "(a+b+c)*(ac*b+bc*a)(a+b+c)*", "c*(a(a+c)*b+b(b+c)*a)(a+b+c)*");

In [ ]:
test(Sigma, "((c*ac*)*(c*ac*)*(c*bc*)(c*bc*)*)+((c*bc*)(c*bc*)*(c*ac*)(c*ac*)*)", "c*(a*(a+c)*b+b*(b+c)*a)(a+b+c)*");

In [ ]:
test(Sigma, "(a+b)*a(a+b)*a(a+b)*a(a+b)*", "a*b*ab*ab*ab*a*");

## 4. Visual Inspection of the Test Cases

Now, let us examine these exact same examples using our **Visual Testing Framework**.

This provides deeper insights:
1.  **For Equivalent Expressions:** We expect to see two minimized DFAs that are **isomorphic** (structurally identical), confirming the result visually.
2.  **For Non-Equivalent Expressions:** We expect the tool to find a concrete **Counter-Example** (the shortest word $w$ that distinguishes the two languages) and display the structural differences between the automata.

In [ ]:
console.log("--- VISUAL RE-RUN ---");

await testVisual(Sigma, '(ε+a)(a+ε)*(a+ε)', 'a*');

await testVisual(Sigma, "(ba)(ba)*", "b(ab)*a");

await testVisual(Sigma, 
    "(a+b+c)*(ac*b+bc*a)(a+b+c)*", 
    "c*(a(a+c)*b+b(b+c)*a)(a+b+c)*"
);

await testVisual(Sigma, 
    "((c*ac*)*(c*ac*)*(c*bc*)(c*bc*)*)+((c*bc*)(c*bc*)*(c*ac*)(c*ac*)*)", 
    "c*(a*(a+c)*b+b*(b+c)*a)(a+b+c)*"
);

await testVisual(Sigma, "(a+b)*a(a+b)*a(a+b)*a(a+b)*", "a*b*ab*ab*ab*a*");